In [18]:
import re
import bs4
import requests

import pymongo

# Request website to scrap

In [3]:
url = "https://www.kdnuggets.com/"

In [4]:
response = requests.get(url)

# Scrap header information and content

In [5]:
# parse response
soup = bs4.BeautifulSoup(response.text, 'html.parser')

# find content
content = soup.find_all('div', {'id': 'content'})

# first 'content' tag contains latest posts, partners' posts, top posts past 30 days, recent posts
tables_content = content[0].find_all('table', {'class': 'thb'})

# first tables content contains latest posts, so only grab tags including the right url
latest_posts = tables_content[0].find_all(href=re.compile(".html"))

In [ ]:
def get_header_info(post_soup: bs4.BeautifulSoup) -> tuple:
    title = post_soup.find('div',{'id':'post-header'}).h1.text.replace('\n', '')
    excerpt = post_soup.find('div',{'id':'post-header'}).p.text.replace('\n', '')
    author = post_soup.find('div', {'id':'post-header'}).a.text.replace('\n', '')

    return (title, excerpt, author)

def get_post_content(post_soup: bs4.BeautifulSoup) -> str:
    tag_texts = []
    
    for tag in post_soup.find('div',{'id':'post-'}):

        if isinstance(tag, bs4.Tag):
            if tag.name in ['div']:
                tag.decompose()
                continue
            if tag.find(['font']):
                tag.decompose()
                continue
            tag_texts.append(tag.text)
    post_content = '\n'.join(tag_texts)
    return post_content

In [23]:
post_scrappings = []

for url_tag in latest_posts:
    url = url_tag['href']
    print(f'Scrapping following url: {url}')

    # request website
    post = requests.get(url)
    post_soup = bs4.BeautifulSoup(post.text, 'html.parser')
    dict_post_scrapping = {}

    # get header info
    title, excerpt, author = get_header_info(post_soup)

    dict_post_scrapping['Title'] = title
    dict_post_scrapping['Excerpt'] = excerpt
    dict_post_scrapping['Author'] = author

    # get post content info
    post_content = get_post_content(post_soup)

    dict_post_scrapping['Content'] = post_content

    # append scrappend information
    post_scrappings.append(dict_post_scrapping)

Scrapping following url: https://www.kdnuggets.com/2023/03/hydra-configs-deep-learning-experiments.html
Scrapping following url: https://www.kdnuggets.com/2023/03/top-free-courses-large-language-models.html
Scrapping following url: https://www.kdnuggets.com/2023/03/time-series-forecasting-statsmodels-prophet.html
Scrapping following url: https://www.kdnuggets.com/2023/03/key-issues-associated-classification-accuracy.html
Scrapping following url: https://www.kdnuggets.com/2023/03/top-posts-week-0227-0305.html
Scrapping following url: https://www.kdnuggets.com/2023/03/getting-started-github-cli.html


In [24]:
print(f'URLs scrapped: {len(post_scrappings)}')

URLs scrapped: 6


# Save data in mongodb

In [26]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client.db.webscrapping
try:
    db.insert_many(post_scrappings)
    print(f'Inserted {len(post_scrappings)} articles')
except:
    print('an error occurred quotes were not stored to db')

Inserted 6 articles
